# РГР: Визначення температурних полів та параметрів масопереносу при імпульсному впливі

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Latex

## Розрахунок температурних полів

**Визначити за даними таблиці значення $\lambda$ \[Дж/см*Kc\] для елементів Cr, Cu, Ni.**

$$\lambda_i = a_i \gamma_i c_i,$$
де:

- $\lambda_i$ - thermal conductivity
- $c_i$ - thermal capacity
- $a_i$ - thermal diffusivity
- $\gamma_i$ - density

In [2]:
df_in = pd.DataFrame(
    {
        'thermal_capacity':    [0.56, 0.47, 0.58],
        'thermal_diffusivity': [0.14, 0.85, 0.16],
        'density':             [8.9,  8.96, 7.1],
        'thickness':           [100]*3
    },
    index=['Ni', 'Cu', 'Cr']
)
df_in['thermal_conductivity'] = df_in.thermal_diffusivity * df_in.density * df_in.thermal_capacity
df_in

,density,thermal_capacity,thermal_diffusivity,thickness,thermal_conductivity
Ni,8.90,0.56,0.14,100,0.69776
Cu,8.96,0.47,0.85,100,3.57952
Cr,7.10,0.58,0.16,100,0.65888


**Розрахувати загальну теплопровідність $\bar{\lambda}$ системи Cr-Cu-Ni:**

$$\bar{\lambda} = \frac{d_1\lambda_1 + d_2\lambda_2 + d_3\lambda_3}{d}$$

In [3]:
sys_thermal_conductivity = (df_in.thermal_conductivity * df_in.thickness).sum() / df_in.thickness.sum()
Latex(r'$\bar{\lambda} = %.2f$' % sys_thermal_conductivity)

<IPython.core.display.Latex object>

**Розрахувати ефективне значення температуропровідності $a_{\textit{пл}}$ плівкової структури уздовж поверхні за формулою:**

$$a_{\textit{пл}} = \frac{\bar{\lambda}}{\bar{c}\bar{\gamma}}$$

In [4]:
sys_thermal_capacity = (df_in.thermal_capacity * df_in.thickness).sum() / df_in.thickness.sum()
sys_density = (df_in.density * df_in.thickness).sum() / df_in.thickness.sum()
efficient_thermal_diffusivity = sys_thermal_conductivity / (sys_thermal_capacity * sys_density)
Latex(r'$a_{\textit{пл}} = %.2f$' % efficient_thermal_diffusivity)

<IPython.core.display.Latex object>

**Визначити температуру плівкової структури Cr-Cu-Ni в епіцентрі під дією лазерного імпульсу для 10 часових інтервалів, на які умовно розбивається тривалість імпульсу, за формулою:**

$$
T = \frac{2E}{\pi^{3/2}\tau}
\frac{1-R}{c\gamma\sqrt{a_c}}
\frac{\sqrt{\Delta t \dot{} n}}{r^2_{\it\Gamma}+ 4a_{\textit{пл}}\Delta t \dot{} n},
$$
де:

- $\tau$ - тривалість імпульсу
- $\Delta t$ величина, яка дорівнює $$\Delta t = \frac{\tau}{n'_{\max}}, \; n'_{\max} = 10$$
- $t$ - час, протягом якого діє джерело тепла: $$t = \Delta t \dot{} n' = \frac{\tau\dot{}n'}{10},$$ $n$ - часовий інтервал ($n' = 1, 2, 3, \dots$)

- Енергія лазерного імпульсу: $E = 99\,\text{мДж}$
- $\frac{1-R}{c\gamma\sqrt{a_c}} = 9.0\,\frac{K\dot{}\text{см}^2\dot{}\text{с}^{1/2}}{\text{Дж}}$
- Тривалість лазерного імпульсу $\tau = 2.56\times10^{-3}\,\text{с}$
- $r_{\it\Gamma} = 0.05\,\text{см}$

In [5]:
tau = 2.56e-3
coeff = 9.0
E = 99e-3
r_batch = 0.05

def calc_temperature(tau, n, E, coeff, r_batch, efficient_thermal_diffusivity):
    Delta_t = tau / 10
    return 2 * E / (np.pi**(1.5) * tau) * \
        coeff * \
        (np.sqrt(Delta_t * n)) / \
        (r_batch**2 + 4*efficient_thermal_diffusivity * Delta_t * n)

N = range(1, 11)
times = np.array([tau / 10 * n for n in N])
temperatures = np.array([calc_temperature(tau, n, E, coeff, r_batch, efficient_thermal_diffusivity) for n in N])

df_out = pd.DataFrame({'time, x10^(-3) s': times * (10**3), 'T, K': temperatures}, index=N)
df_out.round(3)

,"T, K","time, x10^(-3) s"
1,695.138,0.256
2,869.096,0.512
3,953.835,0.768
4,997.735,1.024
5,1019.546,1.280
6,1028.394,1.536
7,1029.268,1.792
8,1025.099,2.048
9,1017.696,2.304
10,1008.221,2.560


## Розрахунок параметрів масопереносу

**Визначити середні коефіцієнти дифузії $D_{\textit{еф}}$ для міді:**
$$D_{\textit{еф}} = \frac{\sum_{j=1}^n D_j t_j}{\tau}$$

Коефіцієнт дифузії розраховується за формулою:

$$D = D_0 \exp{\frac{-E_a}{RT}}$$

In [6]:
D_0 = 1e-3
E = 149

def calc_mass_diffusivity(T, D_0, E):
    return D_0 * np.exp(-E/(8.31*T))

mass_diffusivity = np.apply_along_axis(calc_mass_diffusivity, axis=0, arr=temperatures, D_0=D_0, E=E)
df_out['D'] = mass_diffusivity
df_out

,"T, K","time, x10^(-3) s",D
1,695.137614,0.256,0.000975
2,869.096409,0.512,0.000980
3,953.834616,0.768,0.000981
4,997.734899,1.024,0.000982
5,1019.546189,1.280,0.000983
6,1028.394492,1.536,0.000983
7,1029.268205,1.792,0.000983
8,1025.098592,2.048,0.000983
9,1017.696210,2.304,0.000983
10,1008.220759,2.560,0.000982


In [7]:
efficient_mass_diffusivity = np.sum(mass_diffusivity * times) / tau
Latex(r'$D_{\textit{еф}} = %.2f \times 10^{-3}$' % (efficient_mass_diffusivity * (10**3)))

<IPython.core.display.Latex object>

**Розрахувати середні значення "довжини дифузійного шляху" $x = (Dt)^{1/2}$ за значеннями ефективних коефіцієнтів $D_{\textit{еф}}$**

In [8]:
distance = (efficient_mass_diffusivity * times[-1]) ** (0.5)
Latex(r'$x = %.2f \times10^{-3}\, \text{см}$' % (distance * 10**3))

<IPython.core.display.Latex object>